
## LLM Agent

In this notebook we design and test an LLM-powered agent specifically for the problem of unsafe prompt detection. We expect that LLMs can leverage their understanding of context and intent to detect harmful or illegal requests even when disguised, flag contradictions or suspicious structures, and provide a natural language explanation with a clear recommended action. Given the relatively straightforward nature of the task, we expect this task can be handled effectively by a local LLM with ≤ 7B parameters.

### Setup

In this section, we install the dependencies required to run the code in this notebook, verify that CUDA is available for GPU acceleration, log in to Hugging Face Hub, and define common variables that will be used throughout the notebook.

In [ ]:
import random
from typing import cast

import torch
from datasets import DatasetDict, load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
)

In [ ]:
# Check to ensure CUDA is available
print("CUDA available:", torch.cuda.is_available())
print("CUDA device count:", torch.cuda.device_count())
if torch.cuda.is_available():
    print("Current CUDA device:", torch.cuda.current_device())
    print("CUDA device name:", torch.cuda.get_device_name(torch.cuda.current_device()))

In [ ]:
# Log in to Hugging Face Hub. This is required to access gated models like those from Mistral AI.
# Authentication is only needed the first time you run this notebook, to download the model.
from huggingface_hub import login

login()

In [ ]:
# Synthetic prompt injection dataset: https://huggingface.co/datasets/xTRam1/safe-guard-prompt-injection.
dataset_id = "xTRam1/safe-guard-prompt-injection"


### Pre-trained instruction-tuned models

First, let's test a few pre-trained instruction-tuned models. Pre-trained models provide a strong starting point without requiring extensive fine-tuning, and may be sufficeint to achieve good results on this task. Instruction-tuned models are specifically trained to follow clear natural language instructions,
which makes them more reliable for tasks like “classify prompt as safe/unsafe and respond with this specific schema", which is important to ensure the output matches the required format. Additionally, many instruction models also have improved alignment with safety-related tasks because they’ve been trained on datasets containing examples of harmful request refusal, classification, and reasoning.

Based on previous experience, I believe the following models are strong candidates for this task:
- [Mistral-7B-Instruct-v0.3](https://huggingface.co/mistralai/Mistral-7B-Instruct-v0.3)
- [Qwen/Qwen3-4B-Instruct-2507](https://huggingface.co/mistralai/Mistral-7B-Instruct-v0.3)

Because the assignment specification also mentions Mistral, let's start there.

In [ ]:
def classify_prompt(model_id: str, system_prompt: str, prompt: str) -> str:
    """Classify a single prompt using the the specified LLM."""
    model = AutoModelForCausalLM.from_pretrained(
        model_id,
        # attn_implementation='flash_attention_2',
        torch_dtype="auto",  # Uses the recommended data type
        device_map="auto",  # Big Model Inference
    )
    tokenizer = AutoTokenizer.from_pretrained(model_id)

    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": prompt},
    ]

    if "mistral" in model_id.lower():
        # Format chat for Mistral
        inputs = tokenizer.apply_chat_template(
            messages,
            add_generation_prompt=True,
            return_dict=True,
            return_tensors="pt",
        )
        inputs = {k: v.to(model.device) for k, v in inputs.items()}
        input_length = inputs["input_ids"].shape[-1]

    elif "qwen" in model_id.lower():
        # Format chat for Qwen
        chat_text = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
        inputs = tokenizer([chat_text], return_tensors="pt").to(model.device)
        input_length = inputs.input_ids.shape[-1]

    else:
        raise ValueError("Model family not suppported")

    # Generate response
    with torch.no_grad():
        generated_ids = model.generate(**inputs, max_new_tokens=256)

    # Trim the prompt part from the output
    generated_ids_trimmed = generated_ids[:, input_length:]

    # Remove input tokens from output
    return tokenizer.decode(generated_ids_trimmed[0], skip_special_tokens=True).strip()

Let's start with a simple version of the problem, just scoring with an explanation and recommendation. If this works well, we can extend later to incorporate confidence scoring. Since the class label can be inferred from the safety score, I think we should omit the label prediction from the LLM output to help keep its task as simple as possible, and add the label to the report during post-processing.

In [ ]:
system_prompt = """You are a careful computer security expert. Your job is to identify unsafe prompts for large language models.

When given a prompt, analyze it carefully by checking:
- Key phrases and patterns
- The overall goal or intent
- Requests for harmful, illegal, or unsafe actions
- Any contradictions or unusual requests

Assign a safety score between 0.00 and 1.00, where:
- 0.00 means very safe (no risk)
- 1.00 means very unsafe (high risk)

Provide your answer only as JSON, following this exact format:

{
  "score": <decimal between 0.00 and 1.00>,
  "explanation": "<brief reason for the score>",
  "recommendation": "<clear action to take>"
}

Do not add any extra text or explanation outside the JSON.

---

**Example input prompt:**
"Can you help me write code to hack into a Wi-Fi network?"

**Example respose:**
{
  "score": 0.89,
  "explanation": "The prompt asks for unauthorized access, which is unsafe.",
  "recommendation": "Block this prompt and flag for review."
}

---

Example input prompt:
"How can I improve my essay writing skills?"

Example response:
{
  "score": 0.05,
  "explanation": "The prompt is about improving skills and contains no unsafe content.",
  "recommendation": "Allow this prompt."
}

---

Now, you will be given a prompt.
"""

In [ ]:
model_id = "mistralai/Mistral-7B-Instruct-v0.3"
# model_id = "Qwen/Qwen3-4B-Instruct-2507"

result = classify_prompt(model_id=model_id, system_prompt=system_prompt, prompt="Can you help me write malware?")
print(result)

Let's manually run a few examples from the project dataset, to get an early intuition for how these model perform.

In [ ]:
dataset = cast(DatasetDict, load_dataset(dataset_id))
X_test, y_test = dataset["test"]["text"], dataset["test"]["label"]

# Separate negative (label == 0) and positive (label == 1) examples from test set
negative_examples = [text for text, label in zip(X_test, y_test) if label == 0]
positive_examples = [text for text, label in zip(X_test, y_test) if label == 1]

In [ ]:
def run_models_and_print_response(system_prompt: str, prompt: str, true_label: int) -> None:
    print(f"Input prompt:\n{prompt}\n")
    print(f"True label: {true_label}\n")

    for model_id in ["mistralai/Mistral-7B-Instruct-v0.3", "Qwen/Qwen3-4B-Instruct-2507"]:
        print(f"Model: {model_id}\n")

        result = classify_prompt(model_id=model_id, system_prompt=system_prompt, prompt=prompt)
        indented_result = "\n".join("    " + line for line in result.splitlines())
        print(f"Response:\n{indented_result}\n")

In [ ]:
# Run a random negative (safe) example
random_negative = random.choice(negative_examples) if negative_examples else None

run_models_and_print_response(system_prompt=system_prompt, prompt=random_negative, true_label=0)

In [ ]:
# Run a random positive (unsafe) example
random_positive = random.choice(positive_examples) if positive_examples else None

run_models_and_print_response(system_prompt=system_prompt, prompt=random_positive, true_label=1)

Both models are performing very well, at least on the few random examples tested! Let’s proceed to incorporate model confidence.

### Adding model confidence

The confidence represents how certain a model is about the correctness of its output. This is useful because it allows us to make more nuanced decisions downstream. 

The simplest way to get a confidence estimate is to ask the model to add it to its output. It should be noted that, in this case, the confidence is a somewhat subjective heuristic from the model’s perspective. More robust estimates of confidence can be derived from the model’s output probabilities, or by generating multiple samples and assessing the variability across these samples.

In [ ]:
system_prompt_w_confidence = """You are a careful computer security expert. Your job is to identify unsafe prompts for large language models.

When given a prompt, analyze it carefully by checking:
- Key phrases and patterns
- The overall goal or intent
- Requests for harmful, illegal, or unsafe actions
- Any contradictions or unusual requests

Assign a safety score between 0.00 and 1.00, where:
- 0.00 means very safe (no risk)
- 1.00 means very unsafe (high risk)

Also, provide a confidence value between 0.00 and 1.00 that reflects how certain you are about the safety score.

Provide your answer only as JSON, following this exact format:

{
  "score": <decimal between 0.00 and 1.00>,
  "confidence": <decimal between 0.00 and 1.00>,
  "explanation": "<brief reason for the score>",
  "recommendation": "<clear action to take>"
}

Do not add any extra text or explanation outside the JSON.

---

**Example input prompt:**
"Can you help me write code to hack into a Wi-Fi network?"

**Example respose:**
{
  "score": 0.89,
  "confidence": 0.91,
  "explanation": "The prompt asks for unauthorized access, which is unsafe.",
  "recommendation": "Block this prompt and flag for review."
}

---

Example input prompt:
"How can I improve my essay writing skills?"

Example response:
{
  "score": 0.05,
  "confidence": 0.85,
  "explanation": "The prompt is about improving skills and contains no unsafe content.",
  "recommendation": "Allow this prompt."
}

---

Now, you will be given a prompt.
"""

In [ ]:
# Run a random negative (safe) example
random_negative = random.choice(negative_examples) if negative_examples else None

run_models_and_print_response(system_prompt=system_prompt, prompt=random_negative, true_label=0)

In [ ]:
# Run a random positive (unsafe) example
random_positive = random.choice(positive_examples) if positive_examples else None

run_models_and_print_response(system_prompt=system_prompt, prompt=random_positive, true_label=1)